In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
import numpy as np
from scipy.stats import norm
set_seed(3407)

In [122]:
# Load in UL2 tokenizer to see what's going on
from transformers import AutoTokenizer, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer

PreTrainedTokenizer(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [2]:
dataset = load_dataset("togethercomputer/RedPajama-Data-1T-Sample", 'plain_text', cache_dir='datasets')
dataset = dataset['train']

# Custom dataset class for the Red Pajama dataset
class RedPajamaDataset(Dataset):
    def __init__(self, data, max_length=1024):
        self.data = data
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.add_tokens([f'new_id_{i}' for i in range(200)])
        self.tokenizer.add_tokens(['[S2S]', '[NLU]', '[NLG]'])
        self.tokenizer.pad_token_id = 50256
        self.max_length = max_length
        self.vocab_size = self.tokenizer.vocab_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        # Tokenize the text
        tokens = self.tokenizer.encode(text, add_special_tokens=True, max_length=self.max_length, truncation=True, return_tensors='pt', padding=True)
        # Split the tokens into chunks of max_length
        # Shift the tokens to get targets (excluding the [CLS] token)
        target_tokens = tokens[:, 1:].clone()  # Exclude the [CLS] token
        tokens = tokens[:, :-1]  # Exclude the last token to match the shifted targets
        return tokens, target_tokens
    

# Create an instance of the custom dataset
red_pajama_dataset = RedPajamaDataset(dataset)

Found cached dataset red_pajama-data-1_t-sample (/Users/dylanskinner/Desktop/CS 674 Projects/MinGPT_UL2/datasets/togethercomputer___red_pajama-data-1_t-sample/plain_text/1.0.0/6ea3bc8ec2e84ec6d2df1930942e9028ace8c5b9d9143823cf911c50bbd92039)


  0%|          | 0/1 [00:00<?, ?it/s]

In [124]:
tokenizer.eos_token_id

50256

In [128]:
new_tokens = [f'new_id_{i}' for i in range(200)]
tokenizer.add_tokens(new_tokens)
tokenizer.add_tokens(['[S2S]', '[NLU]', '[NLG]'])

def r_denoising(ids, tokenizer, corruption_pct=0.15, span_length=np.arange(2,6)):
    chance = (corruption_pct / np.mean(span_length)) * (1 + np.max(span_length) / len(ids))
    old_toks = []

    steps_to_skip = 0
    tokens_used = 0

    for i in range(1, len(ids)):
        if steps_to_skip > 0:
            steps_to_skip -= 1
            continue

        if np.random.random() < chance:
            # Get the token we are using for this space
            mask_token = tokenizer.convert_tokens_to_ids(new_tokens[tokens_used])
            tokens_used += 1

            span = np.random.choice(span_length)
            old_toks.append(mask_token)
            old_toks.extend(ids[i:i+span].copy())
            ids = ids[:i] + [mask_token] + ids[i+span:]

            steps_to_skip = span
        
    
    return ids + [tokenizer.eos_token_id] * (1023 - len(ids)), old_toks + [tokenizer.eos_token_id] * (1023 - len(old_toks))

def s_denoising(ids, tokenizer):
    # Get the length of our input
    len_ids = len(ids)

    # Build Gaussian distribution of probabilities for each token
    p = norm.pdf(np.arange(len_ids)/len_ids, loc=np.mean(np.arange(len_ids)/len_ids), scale=np.std(np.arange(len_ids)/len_ids))

    # Normalize the probabilities and get the index: to remove.
    remove_index = np.random.choice(np.arange(len_ids), p=p/p.sum())

    # Get the token we are using for this space
    mask_token = tokenizer.convert_tokens_to_ids(new_tokens[0])

    # Get the tokens we are removing
    old_tok = [mask_token]
    old_tok.extend(ids[remove_index:].copy())

    # Mask the tokens
    ids = ids[:remove_index]
    ids[-1] = mask_token
    return ids + [tokenizer.eos_token_id] * (1023 - len(ids)), old_tok + [tokenizer.eos_token_id] * (1023 - len(old_tok))

def x_denoising(ids, tokenizer, corruption_pct=0.50, span_length=np.arange(2,6)):
    return r_denoising(ids, tokenizer, corruption_pct, span_length)



item = next(iter(dataset))['text']

'''Testing R-Denoising'''
# ids, old_toks = r_denoising(ids, tokenizer, corruption_pct=0.15, span_length=np.arange(2,6))
# old_toks

'''Testing S-Denoising'''
# ids, old_tok = s_denoising(ids, tokenizer)
# tokenizer.decode(ids)

'''Testing X-Denoising'''
# ids, old_toks = x_denoising(ids, tokenizer, corruption_pct=0.50, span_length=np.arange(2,6))
# tokenizer.decode(ids)

'''Testing all at once'''
token_dict = {'s': ['[S2S]', s_denoising], 'r': ['[NLU]', r_denoising], 'x': ['[NLG]', x_denoising]}

# Get the token to prepend and function to use.
begin_id, func = token_dict[np.random.choice(['s', 'r', 'x'], size=1, p=[0.5, 0.25, 0.25])[0]]

# Prepend token to string and tokenize
item = begin_id + ' ' + item
ids = tokenizer(item, truncation=True, max_length=1024)['input_ids']


ids, old_toks = func(ids, tokenizer)
print(ids)
tokenizer.decode(ids)
print('Total non-tokenized', len(ids))
print(f'Total tokenized {len(old_toks)}')
print(f'Total tokens {len(ids) + len(old_toks)}')
print(tokenizer.decode(ids[0]))
tokenizer.decode(old_toks)


# old_toks_dec = [tokenizer.decode(tok) for tok in old_toks]
# old_toks_dec

[50457, 59, 5458, 90, 21906, 92, 198, 59, 18242, 90, 2363, 25, 600, 305, 92, 198, 198, 59, 368, 746, 90, 41394, 9573, 5512, 393, 517, 1690, 663, 3092, 15370, 11, 1871, 6809, 284, 198, 43776, 2478, 4568, 468, 587, 13770, 9713, 287, 2274, 812, 13, 554, 198, 3911, 13174, 11, 262, 4931, 286, 11, 3048, 286, 11, 290, 3753, 47336, 329, 3467, 368, 746, 90, 8388, 198, 220, 10690, 92, 287, 3232, 14, 11505, 8090, 10442, 357, 37, 18420, 8, 423, 2722, 257, 1256, 286, 3241, 198, 2502, 262, 1613, 5707, 93, 59, 66, 578, 90, 67, 8490, 11528, 69, 793, 7959, 82, 11, 10662, 16115, 10333, 74, 67, 413, 3674, 11, 198, 220, 299, 1878, 385, 6999, 8071, 2052, 11, 479, 518, 354, 1754, 6999, 8388, 69, 793, 11, 34439, 2915, 27399, 4967, 8388, 11, 198, 220, 530, 346, 5304, 11275, 1547, 333, 3304, 11, 3857, 829, 5304, 25878, 10745, 793, 11, 1059, 11252, 5539, 8388, 11, 198, 220, 1976, 330, 11072, 3225, 72, 1238, 2481, 8388, 27422, 220, 3467, 368, 746, 90, 10082, 6826, 9573, 92, 318, 319, 262, 584, 1021, 262, 198, 11

'new_id_0  authors during the 1971--2021 time period. \nWe geolocate developers to\n\\DATAWorldRegions/ world regions, using as signals email country code top-level domains (ccTLDs) and \nauthor (first/last) names compared with name distributions around the world, and UTC offsets \nmined from commit metadata.\n\nWe find evidence of the early dominance of North America in open source\nsoftware, later joined by Europe. After that period, the geographic diversity \nin public code has been constantly increasing.\nWe also identify relevant historical shifts\nrelated to the end of the UNIX wars and the increase of coding literacy in\nCentral and South Asia, as well as of broader phenomena like colonialism and\npeople movement across countries (immigration/emigration).\n\n\n\n\n\\paragraph{Data availability.}\n\nA replication package for this paper is available from Zenodo at\n\\url{https://doi.org/10.5281/zenodo.6390355}~\\cite{replication-package}.\n\n\n \\section{Related Work}\n\\label{sec

In [129]:
tokenizer.decode(ids)

'[S2S] \\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontributors 